# Projekt 2 Tweet sentiment analysis

Celem projektu jest stworzenie modelu do oceny czy tweet jest pozytywny, negatywny lub neutralny. W tym celu dokonano przetworzenia otrzymanych tweetów do postaci listy wyrazów, która została ograniczona przez zdefiniowaną listę słów nie oddających emocji. Klasyfkikacji dokonano za pomocą klasyfikatora Naiwnego Bayesa. Dla zbioru testowego otrzymano trafność ponad 56%

Pierwszym krokiem jest import potrzebnych bibliotek.

In [12]:
import re
import nltk
import csv
import pandas as pd

RE_EMOTICONS_POS = re.compile("(:-?\))|(:p)|(:d+)|(;-?\))|(<3)")
RE_EMOTICONS_NEG= re.compile("(:-?\()|(:/)|(=\))|(\)-?:)|(:'\()|(8\))")

Kolejnym krokiem będzie przygotowanie funkcji "czyszczącej" tweety z niepotrzebnych danych czy znaków. Takimi przykładami mogą być wyrazy zaczynające się @ (usunięto ten znak), które odnoszą sie do konkretnego konta Twitter, linki do stron, dodatkowe spacje oraz usunięcie hashtagu (#) sprzed wyrazów. Dodatkowo na samym początku duże litery są zamieniane na małe. Emotikony zgrupowane zostały w pozytywne i negatywne.

In [13]:
def firstClean(text):
    text = str(text)
    text = text.lower()
    text = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','WWW_URL',text)
    
    text = re.sub(RE_EMOTICONS_POS, 'EMO_POS',text)
    text = re.sub(RE_EMOTICONS_NEG, 'EMO_NEG',text)
    
    text = re.sub(r'@([^\s]+)', r'\1',text)
    text = re.sub('[\s]+', ' ', text)
    text = re.sub(r'#([^\s]+)', r'\1', text)
    text = text.strip('\'"?,.')
    
    return text

W celu usunięcia słów, które nie mają nacechowania emocjonalnego lub są częstow używane w j. angielskim użyto listy słów ze strony http://www.ranks.nl/stopwords. Z pobranych słów wcześniej usunięto słowa przeczące (not, arent itp), które mogą mieć znaczenia dla negatywnych tweetów. Dodano również do tej listy znacznik dotyczący adresów URL.

In [14]:
def createStopWordsList():
    stopWords = []
    stopWords.append('WWW_URL')
    
    fp = open('stopWordList.txt', 'r')
    line = fp.readline()
    while line:
        w = line.strip()
        stopWords.append(w)
        line = fp.readline()
    fp.close()
    
    return stopWords

stopWords = []

Pomocnicza funkcja usuwająca ciąg takich samych liter, gdy jest ich więcej niż dwa np looove lub loooove zostanie zastąpnione na loove. 

In [15]:
def deleteTwoPlus(text):
    pattern = re.compile(r"(.)\1{1,}", re.DOTALL)
    return pattern.sub(r"\1\1", text)

Funkcja rozkładająca tweet na osobne wyrazy. Do tworzącej przez tę funkcję listy nie zostają dodane wyrazy zaczynające się od cyfry oraz wyrazy znajdujące się na liście wyrazów odrzuconych (stopWords). Przed porównaniem usuwane są ciągi powtarzających się liter.

In [16]:
def tokenizerTweet(text):
    featureVector = []
    tokens = text.split()
    for t in tokens:
        t = deleteTwoPlus(t)
        val = re.search(r"^[a-z][a-z0-9]*$", t)
        if(t in stopWords or val is None):
            continue
        else:
            featureVector.append(t)
    return featureVector

Funkcja zbierająca dwie funkcje przetwarzająca tweety. Czyszcząca tweet oraz tokenizująca.

In [17]:
def processTweet(text):
    text = firstClean(text)
    return tokenizerTweet(text)

Rozpoczęcie przetwrzania zbioru treningowego. Wywyoływana jest funkcja tworząca listę słów odrzucanych. Każdy tweet ze zbioru treningowego jest przetwarzany oraz tworzona jest lista tokenów na podstawie tokenów z każdego tweetu. Tokeny oraz kategoria tweetu (negative, positive, neutral) dodawane są do listy tweets.

In [18]:
stopWords = createStopWordsList() 

tweets_load = pd.read_table("train.csv",sep = ',')
tweets = []
featureList = []

for index, row in tweets_load.iterrows():
    category = row['Category']
    tweet = row['Tweet']
    vec = processTweet(tweet)
    featureList.extend(vec)
    tweets.append((vec,category))



Dla każdego tweetu tworzona jest lista zawierająca wartości true lub false  w zależności czy występuje w nim wyraz z listy featureList. 

Przykład:
{
    'contains(amendment)': True          
    'contains(bush)': True          
    'contains(scotus)': True 
    .....
}

In [19]:
def extractFeatures(text):
    features = {}
    tweet_words = set(text)
    for word in featureList:
        features['contains(%s)' % word] = (word in tweet_words)
    return features

Z featureList usuwane są duplikaty słów. Następnie dla każdego tweetu treningowego tworzone są vecotry opisane przy funkcji extractFeatures.

In [20]:
featureList = list(set(featureList))
trainingSet = nltk.classify.util.apply_features(extractFeatures, tweets)

Uczymy klasyfikator Naiwnego Bayesa na podstawie zbioru treningowego.

In [ ]:
NBClassifier = nltk.NaiveBayesClassifier.train(trainingSet)

Ostatnim krokiem jest stworzenie pliku wyjściowego dla zbioru testowego, w którym staramy się przewidzieć jakie nacechowanie emocjonalne mają tweety. Opisane kroki doprowadziły do uzyskania ponad 56% trafności dla zbioru testowego.

In [22]:
tweetsTest = pd.read_table("test.csv",sep = ',',dtype={'Id': object})

ofile  = open('result.csv', 'w')
writer = csv.writer(ofile, delimiter=',')

writer.writerow(('Id','Category'))
for index, row in tweetsTest.iterrows():
    if (index < 4000):
        id_n = row['Id']
        tweet = row['Category']
        result = NBClassifier.classify(extractFeatures(processTweet(tweet)))
        writer.writerow((id_n,result))

ofile.close()    